# Import & Load Emergency Care Data (2008 - 2019)

### Config

In [ ]:
import sys
import configparser
config = configparser.ConfigParser()
config.read_file(open('../../settings.ini'))

sys.path.insert(0, config.get('PATHS','libs_path'))
engine_path = config.get('DATABASE','engine_path')

### Main params

In [ ]:
remote_path = ''
local_path = './data_final/'

import postgres #from local file postgres.py
from commons import inline_table_xml, download_file, download_zip_file, extract_zip_file, inline_dimension_xml #from local file commons.py

import json
import pandas as pd
from sqlalchemy import create_engine

### Load file

In [ ]:
#Open unzipped file
df = pd.read_csv("./data_final/fact_emergency.csv", delimiter = ",", dtype = "int")
df.head()

### Ingest

In [ ]:
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(df, 'emergency_care', 'fact_emergency')

### Add Indices to foreign keys

In [ ]:
engine.execute("""
CREATE INDEX fact_emergency_date ON emergency_care.fact_emergency ("date_id"); """)

engine.execute("""
CREATE INDEX fact_emergency_geography ON emergency_care.fact_emergency ("geography_id"); """)

engine.execute("""
CREATE INDEX fact_emergency_emergency ON emergency_care.fact_emergency ("emergency_id"); """)

engine.execute("""
CREATE INDEX fact_emergency_age ON emergency_care.fact_emergency ("age_id"); """)

### Dimensions Table

In [ ]:
import pandas as pd

# DATE
date_df = pd.read_csv("./data_final/date_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(date_df, 'emergency_care', 'date_dimension')

In [ ]:
# GEOGRAPHY
geography_df = pd.read_csv("./data_final/geography_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(geography_df, 'emergency_care', 'geography_dimension')

In [ ]:
# EMERGENCY
emergency_df = pd.read_csv("./data_final/emergency_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(emergency_df, 'emergency_care', 'emergency_dimension')

In [ ]:
# AGE
age_df = pd.read_csv("./data_final/age_table.csv")
engine = create_engine(engine_path)
db = postgres.PostgresDriver(engine)
db.to_sql(age_df, 'emergency_care', 'age_dimension')

In [ ]:
print("All done.")